# Day 5

## Part 1

In [1]:
# lines = [line.rstrip() for line in open('day5_input.txt')]
lines = [line.rstrip() for line in open('day5_sample.txt')]

In [2]:
def split_vals(string):
    return [int(number) for number in string.split(' ') if number.isdigit()]

In [3]:
categories = [(line.split(':')[0], lines.index(line)) for line in lines if ':' in line][1:]
initial_seeds = split_vals(lines[0].split(':')[1])

map_names = []
map_values = []
for i in range(len(categories)):
    start_index = categories[i][1] + 1
    end_index = categories[i+1][1] if i!=len(categories)-1 else -1
    maps = lines[start_index:categories[i+1][1]] if i!=len(categories)-1 else lines[start_index:]
    maps = [line for line in maps if line!='']
    maps = [split_vals(map) for map in maps]
    map_names.append(categories[i][0])
    map_values.append(maps)

In [4]:
destination_range_starts = [[i[0] for i in sublist] for sublist in map_values]
source_range_starts = [[i[1] for i in sublist] for sublist in map_values]
range_lengths = [[i[-1] for i in sublist] for sublist in map_values]

## NAIVE SOLUTION

In [5]:
# maps = []
# for destination_starts, source_starts, ranges in zip(destination_range_starts, source_range_starts, range_lengths):
#     map_dict = {}
#     for destination_start, source_start, r in zip(destination_starts, source_starts, ranges):
#         destinations = [i for i in range(destination_start,destination_start + r)]
#         sources = [i for i in range(source_start, source_start + r)]
#         map_dict.update(dict(zip(sources, destinations)))
#     maps.append(map_dict)

In [6]:
# locations = []
# for seed in initial_seeds:
#     soil = maps[0].get(seed, seed)
#     fertilizer = maps[1].get(soil, soil)
#     water = maps[2].get(fertilizer, fertilizer)
#     light = maps[3].get(water, water)
#     temperature = maps[4].get(light, light)
#     humidity = maps[5].get(temperature, temperature)
#     location = maps[6].get(humidity, humidity)
#     locations.append(location)

# min(locations)

## IMPROVED SOLUTION

In [7]:
maps = []

for destination_starts, source_starts, ranges in zip(destination_range_starts, source_range_starts, range_lengths):
    map_dict = {}
    for destination_start, source_start, r in zip(destination_starts, source_starts, ranges):
        map_dict[source_start] = (destination_start, r)
    maps.append(map_dict)

[seeds, soil, fertilizer, water, light, temperature, humidity] = maps

In [8]:
def lookup(dictionary, i):
    # find the largest key less than or equal to i
    l = [key for key in dictionary if key <= i]
    if len(l) == 0:
        return i
    the_key = max(l)
    mapping_start, mapping_len = dictionary[the_key]
    mapping_offset = i - the_key
    if mapping_offset >= mapping_len:
        return i
    return mapping_start + mapping_offset

def lookup_location(i):
    soil_val = lookup(seeds, i)
    fertilizer_val = lookup(soil, soil_val)
    water_val = lookup(fertilizer, fertilizer_val)
    light_val = lookup(water, water_val)
    temp_val = lookup(light, light_val)
    humidity_val = lookup(temperature, temp_val)
    return lookup(humidity, humidity_val)

In [9]:
locations = [lookup_location(x) for x in initial_seeds]

min(locations)

35

## Part 2

In [10]:
initial_seeds

[79, 14, 55, 13]

In [11]:
seed_starts = initial_seeds[::2]
ranges = initial_seeds[1::2]

## NAIVE SOLUTION

In [12]:
# new_seeds = []
# for seed_start, r in zip(seed_starts, ranges):
#     new_seeds += [i for i in range(seed_start, seed_start + r)]

In [13]:
# locations = [lookup_location(x) for x in new_seeds]
# min(locations)

## Improved solution

In [14]:
def ranges_intersect(a, b):
    # Finds the intersection of two tuples a and b
    # If there is no intersection return None
    if not (a[0] <= b[0] < (a[0] + a[1]) or b[0] <= a[0] < (b[0] + b[1])):
        return None

    if a[0] <= b[0] < (a[0] + a[1]):
        return b[0], min(a[0] + a[1] - b[0], b[1])
    else:
        return a[0], min(b[0] + b[1] - a[0], a[1])

In [15]:
seeds1 = [(seed_start, r) for seed_start, r in zip(seed_starts, ranges)]
soil1 = []
fertilizer1 = []
water1 = []
light1 = []
temperature1 = []
humidity1 = []
location1 = []

In [16]:
all_ranges = []

for destination_starts, source_starts, ranges in zip(destination_range_starts, source_range_starts, range_lengths):
    ranges_ = []
    for destination_start, source_start, r in zip(destination_starts, source_starts, ranges):
        ranges_.append((source_start, r, destination_start))
    all_ranges.append(ranges_)

In [17]:
def process_ranges(prev, next, j):
    for prev_range in prev:
        intersected = []
        for next_range in all_ranges[j]:
            intersect = ranges_intersect((prev_range[0], prev_range[1]), (next_range[0], next_range[1]))
            if intersect is not None:
                intersected.append(intersect)
                next.append((intersect[0] - next_range[0] + next_range[2], intersect[1]))
        intersected.sort(key=lambda x: x[0])
        current_index = prev_range[0]
        for r in intersected:
            if r[0] > current_index:
                next.append((current_index, r[0] - current_index))
            current_index = r[0] + r[1]
        if current_index < prev_range[0] + prev_range[1]:
            next.append((current_index, prev_range[0] + prev_range[1] - current_index))
    print(f'{map_names[j]}: {prev} -> {next}')
    return next

In [18]:
soil1 = process_ranges(seeds1, soil1, 0)
fertilizer1 = process_ranges(soil1, fertilizer1, 1)
water1 = process_ranges(fertilizer1, water1, 2)
light1 = process_ranges(water1, light1, 3)
temperature1 = process_ranges(light1, temperature1, 4)
humidity1 = process_ranges(temperature1, humidity1, 5)
location1 = process_ranges(humidity1, location1, 6)

seed-to-soil map: [(79, 14), (55, 13)] -> [(81, 14), (57, 13)]
soil-to-fertilizer map: [(81, 14), (57, 13)] -> [(81, 14), (57, 13)]
fertilizer-to-water map: [(81, 14), (57, 13)] -> [(81, 14), (53, 4), (61, 9)]
water-to-light map: [(81, 14), (53, 4), (61, 9)] -> [(74, 14), (46, 4), (54, 9)]
light-to-temperature map: [(74, 14), (46, 4), (54, 9)] -> [(45, 11), (78, 3), (82, 4), (90, 9)]
temperature-to-humidity map: [(45, 11), (78, 3), (82, 4), (90, 9)] -> [(46, 11), (78, 3), (82, 4), (90, 9)]
humidity-to-location map: [(46, 11), (78, 3), (82, 4), (90, 9)] -> [(60, 1), (46, 10), (82, 3), (86, 4), (94, 3), (56, 4), (97, 2)]


In [19]:
min([loc[0] for loc in location1])

46